In [ ]:
! pip install neuralprophet --no-index --find-links=file:///kaggle/input/neuralprophet/neuralprophet

# Import Dependencies

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
from neuralprophet import NeuralProphet
from neuralprophet import NeuralProphet, set_log_level
set_log_level("ERROR")
from matplotlib import pyplot as plt
import plotly.express as px
import time
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
# read price df
train="../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv"
test="../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv"
df=pd.read_csv(train)
df1=pd.read_csv(test)
df.head(3)

In [ ]:
df1.head(3)

# Preprocess Data

In [ ]:
df_clean = df.drop(columns=['RowId','SupervisionFlag','Target'], axis=1)
df_clean.head(3)

In [ ]:
df1_clean = df1.drop(columns=['RowId','SupervisionFlag','Target'], axis=1)
df1_clean.head(3)

In [ ]:
df_close = df_clean.drop(columns=['Open','High','Low','Volume','AdjustmentFactor','ExpectedDividend'],axis=1)
df_close.head(3)

In [ ]:
df1_close = df1_clean.drop(columns=['Open','High','Low','Volume','AdjustmentFactor','ExpectedDividend'],axis=1)
df1_close.head(3)

In [ ]:
df_grouped = df_clean.groupby(['SecuritiesCode','Date'])['Close'].sum()

In [ ]:
df1_grouped = df1_clean.groupby(['SecuritiesCode','Date'])['Close'].sum()

In [ ]:
df_grouped = df_grouped.to_frame().reset_index()
df_grouped.head(3)

In [ ]:
df1_grouped = df1_grouped.to_frame().reset_index()
df1_grouped.head(3)

In [ ]:
itemlist = df_clean.SecuritiesCode.unique()
forecast_all = pd.DataFrame()  

# Train and Initiate NeuralProphet

In [ ]:
count=0
for i in itemlist:
    temp = df_grouped[df_grouped.SecuritiesCode == i]
    temp = temp.drop(columns=['SecuritiesCode'])
    temp['Date'] = pd.to_datetime(temp['Date'])
    temp = temp.set_index('Date')
    d_df = temp.resample('D').sum()
    d_df = d_df.reset_index().dropna()
    d_df.columns = ['ds','y']
    d_df = d_df[d_df['ds'].dt.dayofweek < 5]
    
    temp1 = df1_grouped[df1_grouped.SecuritiesCode == i]
    temp1 = temp1.drop(columns=['SecuritiesCode'])
    temp1['Date'] = pd.to_datetime(temp1['Date'])
    temp1 = temp1.set_index('Date')
    d_df1 = temp1.resample('D').sum()
    d_df1 = d_df1.reset_index().dropna()
    d_df1.columns = ['ds','y']
    d_df1 = d_df1[d_df1['ds'].dt.dayofweek < 5]
  
    m = NeuralProphet(epochs=5,learning_rate=2.5)
    
    # create a test holdout set:
    df_train_val, df_test = m.split_df(df=d_df1, freq="D", valid_p=0.2)
    # create a validation holdout set:
    df_train, df_val = m.split_df(df=d_df, freq="D", valid_p=0.2)
    
    # fit a model on training data and evaluate on validation set.
    metrics_train1 = m.fit(df=df_train, freq="D")
    metrics_val = m.test(df=df_val)
    
    # refit model on training and validation data and evaluate on test set.
    m = NeuralProphet(epochs=5,learning_rate=2.5)
    metrics_train2 = m.fit(df=df_train_val, freq="D")
    metrics_test = m.test(df=df_test)
    
    m = NeuralProphet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            epochs=15,
            learning_rate=2.5,
        )
    metrics = m.fit(d_df,freq='D')
    
    future = m.make_future_dataframe(d_df, periods=87, n_historic_predictions=13)
    forecast = m.predict(future)
    forecast['SecuritiesCode'] = i
    forecast_all = pd.concat((forecast_all, forecast))
   
    count +=1
    print(f'Securities Code: {i}, Loop Iteration: {count}')

# Clean Forecast Results

In [ ]:
# read price df
df = forecast_all
df = df[df['ds'].dt.dayofweek < 5]
df.tail(3)

In [ ]:
# getting close prices for all SecuritiesCodes
close_prices = df[['SecuritiesCode','ds','yhat1']]
close_prices['Close'] = close_prices['yhat1']
close_prices['Date'] = close_prices['ds']
close_prices = close_prices.drop(columns=['yhat1','ds'],axis=1)
close_prices.head(3)

In [ ]:
trading_dates = np.array(sorted(close_prices['Date'].unique()))
stock_ids = np.array(sorted(close_prices['SecuritiesCode'].unique()))

temp_mat = pd.DataFrame(np.nan, index=stock_ids, columns=trading_dates)

def create_factor(item, temp_mat=temp_mat):
    output_mat = pd.pivot_table(close_prices,
                                values=item,
                                index='SecuritiesCode', columns='Date')
    
    output_factor = temp_mat.copy()
    output_factor.loc[output_mat.index, output_mat.columns] = output_mat.values

    return output_factor

prices_target = create_factor('Close')
prices_target.head(3)

In [ ]:
stock_return = prices_target.pct_change(1,axis=1)
stock_return.head(3)

In [ ]:
stock_return.iloc[:,0] = stock_return.iloc[:,0].fillna(0)
stock_return.head(3)

# Competition Submission

In [ ]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

In [ ]:
counter=0

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    date = prices['Date'][0]
    print(" => {} Now generating ranking at date [{}]".format(counter+1, date))
    stocks = sample_prediction['SecuritiesCode'].values

    # create prediction df 
    prediction = stock_return.loc[stocks,date]

    # convert to rank
    ranked = prediction.rank(method='first',ascending=False, na_option='bottom')
    column_names =  ['SecuritiesCode', 'Rank']
    ranked_df = ranked.to_frame()
    ranked_df = ranked_df.reset_index()
    ranked_df.columns = column_names
    
    # add rank to df
    feature_map = ranked_df.set_index('SecuritiesCode')['Rank'].to_dict()
    sample_prediction['Rank'] = sample_prediction['SecuritiesCode'].map(feature_map)
    
    #### upload prediction ####
    assert sample_prediction.Rank.nunique() == 2000, 'duplicate rank'
    assert sample_prediction["Rank"].notna().all(), 'na value'
    assert sample_prediction["Rank"].min() == 0, 'rank below 0'
    assert sample_prediction["Rank"].max() == len(sample_prediction["Rank"]) - 1, 'rank above 1999'
    
    sample_prediction['Rank'] = np.int64(sample_prediction['Rank'].values-1)
    env.predict(sample_prediction[['Date', 'SecuritiesCode', 'Rank']])
    ########################

    counter+=1

In [ ]:
sample_prediction.info()

In [ ]:
sample_prediction

In [ ]:
! head submission.csv